In [27]:
import sys
sys.path.insert(1, "../..")

from utils import llm_api
import numpy as np
import random
import string
import pandas as pd
from tasks.graph import graph_utils
import json
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

In [43]:
MODELS = [
    "llama3.1:8b",
    "mistral:instruct",
    "qwen2:7b",
    "deepseek-coder-v2:16b"
]
MODEL = "qwen2:7b"
GOD_MODEL = "gpt-4o-mini"

In [44]:
data = pd.read_csv("../../data/graph/graph_results.csv")
data.head()

,model,edge_size,algorithm,vertex,diff,original_prompt,generated_prompt,god_model
0,gemma2:9b,56,naive,0,0,"[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...","[[0, 18], [0, 3], [0, 5], [0, 10], [0, 15], [0...",gpt-3.5-turbo
1,gemma2:9b,56,opt,0,0,"[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...","[[0, 18], [0, 3], [0, 5], [0, 10], [0, 15], [0...",gpt-3.5-turbo
2,gemma2:9b,56,no_opt,0,1,"[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...","[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...",gpt-3.5-turbo
3,gemma2:9b,56,naive,5,1,"[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...","[[0, 5], [5, 17], [3, 5], [5, 19], [5, 15], [5...",gpt-3.5-turbo
4,gemma2:9b,56,opt,5,0,"[[0, 18], [16, 17], [10, 13], [11, 18], [1, 7]...","[[0, 5], [5, 17], [3, 5], [5, 19], [5, 15], [5...",gpt-3.5-turbo


In [45]:
data = data[data["model"] == MODEL]
data = data[data["god_model"] == GOD_MODEL]

exposure_errors = []
normal_errors = []

for i, row in tqdm(data.iterrows(), total=data.shape[0]):
    generated_prompt = json.loads(row["generated_prompt"])
    
    if row["algorithm"] == "no_opt":
        random.shuffle(generated_prompt)
    if row["algorithm"] != "opt":
        continue

    # Normal
    questions = [
        "The following is a graph given as a list of edges:",
        "\n".join([str(e) for e in generated_prompt]),
        f"What is the degree of node {row['vertex']}? Answer with a number without furthur explanations."
    ]
    response = llm_api.ask(questions, GOD_MODEL)
    response = llm_api.take_out_number(response)
    gt = graph_utils.get_degree(generated_prompt, row["vertex"])
    normal_error = abs(gt - response)
    normal_errors.append(normal_error)

    # Reversed
    # window = len(generated_prompt) // 10
    # generated_prompt[:window], generated_prompt[window:2*window] = generated_prompt[window:2*window], generated_prompt[:window]
    
    questions = [
        "The following is a graph given as a list of edges:",
        "\n".join([str(e) for e in generated_prompt]),
        f"What is the degree of node {row['vertex']}? Answer with a number without furthur explanations."
    ]
    response = llm_api.ask(questions, GOD_MODEL)
    response = llm_api.take_out_number(response)
    gt = graph_utils.get_degree(generated_prompt, row["vertex"])
    exposure_error = abs(gt - response)
    exposure_errors.append(exposure_error)

  0%|          | 0/560 [00:00<?, ?it/s]

100%|██████████| 560/560 [02:21<00:00,  3.97it/s]


In [46]:
data = data[data["algorithm"] == "opt"]
data = data[["edge_size", "diff", "vertex"]]
data["normal_error"] = normal_errors
data["exposure_error"] = exposure_errors

In [47]:
tmp = data.groupby("edge_size").mean().reset_index()
tmp.head()

,edge_size,diff,vertex,normal_error,exposure_error
0,56,1.000000,7.5,0.950000,1.300000
1,104,1.533333,12.5,1.433333,0.666667
2,200,3.150000,17.5,4.900000,1.250000
3,504,14.820000,22.5,22.700000,3.700000


In [41]:
tmp = data.groupby("edge_size").mean().reset_index()
tmp.head()

,edge_size,diff,vertex,normal_error,exposure_error
0,56,0.850000,7.5,0.550000,0.80
1,104,0.966667,12.5,0.733333,0.80
2,200,0.850000,17.5,0.875000,1.95
3,504,8.140000,22.5,4.820000,6.28
